In [2]:
titanic = pd.read_csv("../train.csv")
titanic.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


In [5]:
df1 = titanic.groupby(["Sex","Pclass"]).size().reset_index(name="Counts")
df1

,Sex,Pclass,Counts
0,female,1,94
1,female,2,76
2,female,3,144
3,male,1,122
4,male,2,108
5,male,3,347


##### pivot 사용하여 아래와 같이 만들기
```
등급   1    2    3
성별
-----------------
female 000  000  000
male   000  000  000
```

In [4]:
df1 = df1.pivot("Sex","Pclass","Counts")
df1

Pclass,1,2,3
Sex,,,
female,94,76,144
male,122,108,347


```
생존   0    1
성별
-----------------
female 000  000  
male   000  000
```

In [6]:
df2 = titanic.groupby(["Sex","Survived"]).size().reset_index(name="Counts")
df2

,Sex,Survived,Counts
0,female,0,81
1,female,1,233
2,male,0,468
3,male,1,109


In [7]:
df2 = df2.pivot("Sex","Survived","Counts")
df2

Survived,0,1
Sex,,
female,81,233
male,468,109


In [8]:
# 객실 등급에 따른 생존자수
df3 = titanic.groupby(["Pclass","Survived"]).size().reset_index(name="Counts")
df3

,Pclass,Survived,Counts
0,1,0,80
1,1,1,136
2,2,0,97
3,2,1,87
4,3,0,372
5,3,1,119


In [9]:
df3 = df3.pivot("Pclass","Survived","Counts")
df3

Survived,0,1
Pclass,,
1,80,136
2,97,87
3,372,119


### pivot table

In [10]:
# 객실 등급에 따른 남녀 승객 수
# 1. 새로운 column "Counts"를 만든다.
titanic["Counts"] = 1
titanic.tail(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Counts
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.45,NaN,S,1
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.00,C148,C,1
890,891,0,3,"Dooley, Mr. Patrick",male,32.0,0,0,370376,7.75,NaN,Q,1


In [12]:
# 2. pivot table을 만든다. (value, index, column, aggfunc='value값 어떻게 표현할지 결정')
titanic.pivot_table("Counts","Pclass","Sex", aggfunc=np.sum)

Sex,female,male
Pclass,,
1,94,122
2,76,108
3,144,347


In [13]:
# 성별에 따른 생존자 수
titanic.pivot_table("Counts","Sex","Survived", aggfunc=np.sum)

Survived,0,1
Sex,,
female,81,233
male,468,109


In [14]:
# 성별과 객실등급에 따른 생존자수
titanic.pivot_table("Counts",["Sex","Pclass"],"Survived", aggfunc=np.sum)

Survived         0   1
Sex    Pclass         
female 1         3  91
       2         6  70
       3        72  72
male   1        77  45
       2        91  17
       3       300  47

##### total

In [15]:
df = titanic.pivot_table("Counts","Survived","Sex", aggfunc=np.sum)
df

Sex,female,male
Survived,,
0,81,468
1,233,109


In [16]:
# column total
df["Total"] = df["female"] + df["male"]
df

Sex,female,male,Total
Survived,,,
0,81,468,549
1,233,109,342


In [17]:
# row total
df.loc["total"] = df.loc[0] + df.loc[1]
df

Sex,female,male,Total
Survived,,,
0,81,468,549
1,233,109,342
total,314,577,891


In [18]:
# 삭제하기~
df.drop("total", axis=0, inplace=True)
df

Sex,female,male,Total
Survived,,,
0,81,468,549
1,233,109,342


In [19]:
df.drop("Total", axis=1, inplace=True)
df

Sex,female,male
Survived,,
0,81,468
1,233,109


In [21]:
df3 = titanic.pivot_table("Counts","Survived",["Parch","Pclass"],\
                         aggfunc=np.sum, fill_value=0)
df3

Parch      0            1           2          3     4     5  6
Pclass     1   2    3   1   2   3   1   2   3  2  3  1  3  3  3
Survived                                                       
0         64  86  295  10   8  35   5   3  32  0  2  1  3  4  1
1         99  48   86  21  24  20  16  13  11  2  1  0  0  1  0

In [23]:
df4 = titanic.pivot_table("Counts","Survived",["Parch","Pclass"],\
                         aggfunc=np.sum, dropna=False, fill_value=0)
df4

Parch      0            1           2          3 ...     4        5        6  \
Pclass     1   2    3   1   2   3   1   2   3  1 ...  3  1  2  3  1  2  3  1   
Survived                                         ...                           
0         64  86  295  10   8  35   5   3  32  0 ...  2  1  0  3  0  0  4  0   
1         99  48   86  21  24  20  16  13  11  0 ...  1  0  0  0  0  0  1  0   

Parch           
Pclass    2  3  
Survived        
0         0  1  
1         0  0  

[2 rows x 21 columns]